# Setup

In [ ]:
import os
import logging as log
from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, trim_wa_path, ptable

hv.extension('bokeh')
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
hv.renderer('bokeh').webgl = True
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

## Runs

In [ ]:
drarm = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/drarm/', [
    'drarm_product_baseline_10_1407',
    'drarm_product_no_vh_sched_adpf_10_1107',
    'drarm_product_arm_vh_sched_stub_10_1307',
])

drarm.show()

# FPS Report

In [ ]:
def preproces_adpf(df):
    return df.reset_index().rename(columns={'index':'ts'})

drarm.load_combined_analysis('adpf.pqt', preprocess=preproces_adpf)
drarm.load_combined_analysis('adpf_totals.pqt')
drarm.analysis['adpf_totals_melt'] = pd.melt(drarm.analysis['adpf_totals'], id_vars=['iteration', 'wa_path'], value_vars=['average fps', 'frame count'])

ds_adpf = hv.Dataset(drarm.analysis['adpf'].reset_index(), ['ts', hv.Dimension('wa_path', values=drarm.wa_paths)], [
    'average fps', 'sigma fps', 'thermal status',
    'Adaptive Batching', 'sn_Adaptive Batching', 'Adaptive Decals',
    'sn_Adaptive Decals', 'Adaptive Framerate', 'sn_Adaptive Framerate',
    'Adaptive LOD', 'sn_Adaptive LOD', 'Adaptive Lut', 'sn_Adaptive Lut',
    'Adaptive MSAA', 'sn_Adaptive MSAA', 'Adaptive Resolution',
    'sn_Adaptive Resolution', 'Adaptive Shadow Cascade',
    'sn_Adaptive Shadow Cascade', 'Adaptive Shadow Distance',
    'sn_Adaptive Shadow Distance', 'Adaptive Shadowmap Resolution',
    'sn_Adaptive Shadowmap Resolution', 'Adaptive Shadow Quality',
    'sn_Adaptive Shadow Quality', 'Adaptive Transparency',
    'sn_Adaptive Transparency', 'Adaptive View Distance',
    'sn_Adaptive View Distance', 'Adaptive Sorting', 'sn_Adaptive Sorting',
    'Adaptive Physics', 'sn_Adaptive Physics', 'Adaptive Layer Culling',
    'sn_Adaptive Layer Culling', 'Adaptive Fog', 'sn_Adaptive Fog',
])

drarm.analysis['adpf_totals_melt']

## Per-iteration FPS line plot

In [ ]:
drarm.plot_lines_px(drarm.analysis['adpf_totals_melt'], facet_col='variable')

## FPS & total frames bar plot

In [ ]:
drarm.summary['fps'] = drarm.plot_gmean_bars(drarm.analysis['adpf_totals_melt'], x='metric', y='value', facet_col='variable', facet_col_wrap=5, title='Gmean iteration average FPS & total frames', height=600, include_columns=['variable'], table_sort=['variable', 'kernel'])

## FPS over time

In [ ]:
layout = ds_adpf.to(hv.Curve, 'time', 'average fps').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False, title='Average FPS')
layout += ds_adpf.to(hv.Curve, 'time', 'sigma fps').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False, title='Sigma FPS')
layout = layout.cols(1)
layout.opts(
    opts.Curve(height=600, width=1200, framewise=True),
)
layout

# Overutilized

In [ ]:
def postprocess_overutil(df):
    df['time'] = round(df['time'], 2)
    df['total_time'] = round(df['total_time'], 2)
    return df

drarm.load_combined_analysis('overutilized.pqt', postprocess=postprocess_overutil)
drarm.load_combined_analysis('overutilized_mean.pqt')
ptable(drarm.analysis['overutilized_mean'])

## Line plot

In [ ]:
drarm.plot_lines_px(drarm.analysis['overutilized'], y='percentage', title='Overutilized percentage per-iteration')

# Power meter

In [ ]:
def postprocess_pixel6_emeter_means(df):
    df_total = df.groupby(['wa_path', 'kernel', 'iteration']).sum(numeric_only=True).reset_index()
    df_total['channel'] = 'Total'

    df_cpu_total = df.query("channel.str.startswith('CPU')").groupby(['wa_path', 'kernel', 'iteration']).sum(numeric_only=True).reset_index()
    df_cpu_total['channel'] = 'CPU'
    return pd.concat([df, df_cpu_total, df_total])[['wa_path', 'kernel', 'iteration', 'channel', 'power']]


drarm.load_combined_analysis('pixel6_emeter.pqt')
drarm.load_combined_analysis('pixel6_emeter_mean.pqt', postprocess=postprocess_pixel6_emeter_means)

display(drarm.analysis['pixel6_emeter_mean'])

## Line plot across iterations

In [ ]:
drarm.plot_lines_px(drarm.analysis['pixel6_emeter_mean'], y='power', facet_col='channel', facet_col_wrap=3, height=1000, title='Mean power usage across iterations [mW]')

## Bar plot - gmean

In [ ]:
drarm.summary['power_usage'] = drarm.plot_gmean_bars(drarm.analysis['pixel6_emeter_mean'].rename(columns={'power':'value'}), x='channel', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean power usage [mW]', height=600, include_total=True, include_columns=['channel'])

# Frequency

In [ ]:
def postprocess_freq(df):
    df['unit'] = 'MHz'
    df['metric'] = 'frequency'
    df['order'] = df['cluster'].replace('little', 0).replace('mid', 1).replace('big', 2)
    return df.sort_values(by=['iteration', 'order']).rename(columns={'frequency':'value'})

drarm.load_combined_analysis('freqs_mean.pqt', postprocess=postprocess_freq)
display(drarm.analysis['freqs_mean'].head())

## Line plot

In [ ]:
ds = hv.Dataset(drarm.analysis['freqs_mean'], ['iteration', hv.Dimension('wa_path', values=drarm.wa_paths), hv.Dimension('cluster', values=drarm.CLUSTERS)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster frequency across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='MHz'),
)
layout

## Bar plot

In [ ]:
drarm.summary['frequency'] = drarm.plot_gmean_bars(drarm.analysis['freqs_mean'], x='metric', y='value', facet_col='cluster', facet_col_wrap=3, title='Gmean frequency per cluster', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# Capacity

In [ ]:
drarm.load_combined_analysis('capacity.pqt', allow_missing=True)

display(drarm.analysis['capacity'])

In [ ]:
ds = hv.Dataset(drarm.analysis['capacity'].query("iteration == 7 and cpu == 6"), ['time_it', hv.Dimension('wa_path', values=drarm.wa_paths), 'iteration', 'cpu'], ['capacity'])
layout_cap = ds.to(hv.Curve, 'time_it', 'capacity').overlay('wa_path').opts(width=2000, height=600)
layout_cap

# uclamp

In [ ]:
def postprocess_uclamp_updates(df):
    return df.query("task != '<unknown>'")

drarm.load_combined_analysis('uclamp_updates.pqt', postprocess=postprocess_uclamp_updates, allow_missing=True)

display(drarm.analysis['uclamp_updates'])

## Per-task uclamp over time

In [ ]:
ds = hv.Dataset(drarm.analysis['uclamp_updates'], ['time_it', hv.Dimension('wa_path', values=drarm.wa_paths), 'iteration', 'uclamp_id', 'task'], ['value'])
layout = ds.to(hv.Curve, 'time_it', 'value').overlay('wa_path').opts(shared_axes=False, title='Per-task uclamp over time')

layout.opts(
    opts.Curve(height=600, width=1600, interpolation='steps-post', framewise=True)
)
layout

## Frequency & uclamp over time

In [ ]:
drarm.load_combined_analysis('freqs.pqt')

def adjust_df_trace_time(it, path, df):
    df['Time'] = df['Time'] - drarm.traces[path][it].start
    return df

drarm.analysis['freqs_adj'] = pd.concat([adjust_df_trace_time(tags[0], tags[1], df) for tags, df in drarm.analysis['freqs'].groupby(["iteration", 'wa_path'])])
drarm.analysis['freqs_adj']['frequency_scaled'] = drarm.analysis['freqs_adj']['frequency'] / 100000

display(drarm.analysis['freqs_adj'])

In [ ]:
layout = hv.Dataset(drarm.analysis['uclamp_updates'].query("uclamp_id == 0"), ['Time', hv.Dimension('wa_path', values=drarm.wa_paths), 'iteration'], ['value']).select(
    task='UnityMain'
).to(hv.Curve, 'Time', 'value').overlay('wa_path').opts(shared_axes=False, title='CPU frequency & per-task uclamp over time')
layout *= hv.Dataset(drarm.analysis['freqs_adj'], ['Time', hv.Dimension('wa_path', values=drarm.wa_paths), 'cpu', 'iteration'], ['frequency_scaled']).to(
    hv.Curve, 'Time', 'frequency_scaled'
).overlay('wa_path').opts(shared_axes=False)

layout.opts(
    opts.Curve(height=600, width=1600, interpolation='steps-post', framewise=True)
)
layout

# Task placement (activations)

## Overview

In [ ]:
# select all tasks specified in the config by default
plot_tasks = drarm.config['processor']['important_tasks']['drarm'].get()
# override to select fewer tasks - must be a subset of the above
plot_tasks = ['UnityMain', 'UnityGfxDeviceW', 'UnityChoreograp', 'RenderEngine', 'surfaceflinger', 'mali_jd_thread']

def postprocess_task_activations_stats_cluster_df(df):
    return df.query("comm in @plot_tasks").reset_index(drop=True)

drarm.load_combined_analysis('task_activations_stats_cluster.pqt', postprocess=postprocess_task_activations_stats_cluster_df)
drarm.analysis['task_activations_stats_cluster_melt'] = pd.melt(drarm.analysis['task_activations_stats_cluster'], id_vars=['kernel', 'wa_path', 'iteration', 'cluster', 'comm'], value_vars=['count', 'duration'])
drarm.analysis['task_activations_stats_cluster_melt']

### Line plots - counts

In [ ]:
for task, task_df in drarm.analysis['task_activations_stats_cluster'].groupby('comm'):
    drarm.plot_lines_px(task_df, x='iteration', y='count', color='wa_path', facet_col='cluster', facet_col_wrap=3, height=500, scale_y=True, title=f'Activations of {task} per cluster across iterations')

### Line plots - durations

In [ ]:
for task, task_df in drarm.analysis['task_activations_stats_cluster'].groupby('comm'):
    drarm.plot_lines_px(task_df, x='iteration', y='duration', color='wa_path', facet_col='cluster', facet_col_wrap=3, height=500, scale_y=True, title=f'Activation durations of {task} per cluster across iterations')

### Bar plot - counts

In [ ]:
drarm.summary['activations_stats_count'] = drarm.plot_gmean_bars(
    drarm.analysis['task_activations_stats_cluster_melt'].query("variable == 'count'").reset_index(drop=True),
    x='cluster', facet_col='comm', facet_col_wrap=3, title='Gmean task activation counts', height=1000, include_columns=['cluster'], order_cluster=True, percentage=False
)

### Bar plot - durations

In [ ]:
drarm.summary['activations_stats_durations'] = drarm.plot_gmean_bars(
    drarm.analysis['task_activations_stats_cluster_melt'].query("variable == 'duration'").reset_index(drop=True),
    x='cluster', facet_col='comm', facet_col_wrap=3, title='Gmean task activation durations', height=3000, include_columns=['cluster'], order_cluster=True, percentage=False
)

## Detailed plots

### Run setup

In [ ]:
TASK_PL_CMP_WA_PATH_A = drarm.wa_paths[0]
TASK_PL_CMP_WA_PATH_B = drarm.wa_paths[1]
TASK_PL_CMP_IT = 1

def plot_task_activations(wa_path_a, wa_path_b, iteration, comm):
    plot_a = drarm.traces[wa_path_a][iteration].ana.tasks.plot_tasks_activation(drarm.traces[wa_path_a][iteration].get_task_ids(comm)).opts(
        title=f'{comm} activations in iteration {iteration} of {wa_path_a}'
    )
    
    plot_b = drarm.traces[wa_path_b][iteration].ana.tasks.plot_tasks_activation(drarm.traces[wa_path_b][iteration].get_task_ids(comm)).opts(
        title=f'{comm} activations in iteration {iteration} of {wa_path_b}'
    )
    
    return (plot_a + plot_b).cols(1).opts(shared_axes=False)
    

### UnityMain

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'UnityMain')

### UnityGfxDeviceW

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'UnityGfxDeviceW')

### UnityChoreograp

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'UnityChoreograp')

### Thread-7

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'Thread-7')

### Thread-6

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'Thread-6')

### Thread-5

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'Thread-5')

### Thread-4

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'Thread-4')

### surfaceflinger

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMP_IT, 'surfaceflinger')

### mali-cmar-backe

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'mali-cmar-backe')

### mali_jd_thread

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'mali_jd_thread')

### writer

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'writer')

### FastMixer

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'FastMixer')

### RenderEngine

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'RenderEngine')

### Audio Mixer Thr

In [ ]:
plot_task_activations(TASK_PL_CMP_WA_PATH_A, TASK_PL_CMP_WA_PATH_B, TASK_PL_CMD_IT, 'Audio Mixer Thr')

# ADPF

## Thermal status

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'thermal status').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False, title='Thermal status')
layout.opts(
    opts.Curve(height=600, width=1200),
)
layout

## Adaptive Batching

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Batching').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Batching').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Decals

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Decals').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Decals').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Framerate

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Framerate').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Framerate').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive LOD

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive LOD').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive LOD').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Lut

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Lut').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Lut').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive MSAA

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive MSAA').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive MSAA').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Resolution

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Resolution').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Resolution').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Shadow Cascade

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Shadow Cascade').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Shadow Cascade').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Shadow Distance

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Shadow Distance').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Shadow Distance').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Shadowmap Resolution

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Shadowmap Resolution').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Shadowmap Resolution').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Shadow Quality

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Shadow Quality').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Shadow Quality').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Transparency

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Transparency').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Transparency').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive View Distance

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive View Distance').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive View Distance').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Sorting

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Sorting').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Sorting').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Physics

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Physics').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Physics').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Layer Culling

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Layer Culling').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Layer Culling').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout

## Adaptive Fog

In [ ]:
layout = ds_adpf.to(hv.Curve, 'ts', 'Adaptive Fog').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout += ds_adpf.to(hv.Curve, 'ts', 'sn_Adaptive Fog').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False)
layout.opts(
    opts.Curve(height=600, width=1000),
)
layout